In [1]:
!pip install requests requests_cache retry_requests openmeteo_requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.8 MB/s eta 0:00:00


In [2]:
import requests

def get_user_location():
    """
    Fetch the user's approximate location using IP-based geolocation.
    """
    try:
        response = requests.get("https://ipinfo.io/json")
        data = response.json()

        if "loc" in data:
            lat, lon = map(float, data["loc"].split(","))
            return lat, lon, data["city"]
        else:
            raise ValueError("Could not fetch location.")

    except requests.RequestException as e:
        raise ValueError(f"Error fetching location: {e}")

# Example Usage
lat, lon, city = get_user_location()
print(f"Detected Location: {city} ({lat}, {lon})")


Detected Location: Las Vegas (36.175, -115.1372)


In [12]:
import openmeteo_requests
import requests_cache
from retry_requests import retry
import requests

# Open-Meteo API setup
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# IP Stack API Setup (Replace with your actual API key)
API_KEY = "e5f50a554cd6931ae7a4a59de75cdd25"
ipstack_url = f"http://api.ipstack.com/check?access_key={API_KEY}"

def get_user_location():
    """
    Fetch the user's current location (latitude, longitude) and city using IP Stack.
    """
    try:
        # Send request to ipstack
        response = requests.get(ipstack_url)
        data = response.json()

        # Extract location details
        lat = data.get('latitude')
        lon = data.get('longitude')
        city = data.get('city', 'Unknown')

        return lat, lon, city
    except Exception as e:
        print(f"Error fetching location: {e}")
        return None, None, None

def get_weather_data(lat, lon):
    """
    Fetch current weather data from Open-Meteo API using latitude and longitude.
    """
    # Open-Meteo API URL (No API key required)
    url = "https://api.open-meteo.com/v1/forecast"

    # Define parameters for current weather and hourly weather data
    params = {
        "latitude": lat,
        "longitude": lon,
        "current_weather": True,  # Fetch current weather
        "hourly": ["relative_humidity_2m", "apparent_temperature"]  # Get hourly humidity and temperature
    }

    # Fetch data
    responses = openmeteo.weather_api(url, params=params)

    # Process the response
    response = responses[0]

    # Extract weather data
    current = response.Current()
    humidity = current.Variables(0).Value()  # Humidity
    temperature = current.Variables(1).Value()  # Temperature

    return humidity, temperature

# Step 1: Get user's location
lat, lon, city = get_user_location()

if lat is not None and lon is not None:
    print(f"Detected Location: {city} ({lat}, {lon})")

    # Step 2: Fetch weather data based on location
    humidity, temperature = get_weather_data(lat, lon)

    # Step 3: Print the fetched data
    print(f"Weather Data for {city}:")
    print(f" - Temperature: {temperature}°C")
    print(f" - Humidity: {humidity}%")
else:
    print("Could not retrieve location. Please check your connection or IP.")


Detected Location: Sunrise Manor (36.142398834228516, -115.224853515625)
Weather Data for Sunrise Manor:
 - Temperature: 18.356470108032227°C
 - Humidity: 9.170999526977539%
